In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.dummy import DummyRegressor

In [75]:
data_orig = pd.read_csv('../MA_PREDICTOR/data/ma_detailed_data_car_clean.csv')

In [76]:
data = data_orig[(data_orig.car > -0.3) & (data_orig.car < 0.3)]

In [77]:
data.drop(columns='announcement_date', inplace=True)

In [78]:
y=data['car']

In [79]:
X=data[[ 'target_status', 'shares_at_announcement',
       'shares_acquired', 'consideration_offered', 'month', 'rel_deal_value',
       'cross_border', 'relatedness', 'economic_sector_ac',
       'business_sector_ac', 'economic_sector_target',
       'business_sector_target', 'a_fin_adv_count', 't_fin_adv_count']]

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

# Baseline

In [83]:
baseline_model_mean = DummyRegressor(strategy="mean") 

In [84]:
cross_val_score(baseline_model_mean, X_train, y_train, cv=5, scoring='r2').mean()

-0.0029046994417758487

In [85]:
cross_val_score(baseline_model_mean, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

-0.0024689986443534014

# Pipeline

In [86]:
cat_transformer = OneHotEncoder(handle_unknown='ignore')

In [87]:
cat_features=['target_status', 'shares_at_announcement',
       'shares_acquired', 'consideration_offered',
       'cross_border', 'relatedness', 'economic_sector_ac',
       'business_sector_ac', 'economic_sector_target',
       'business_sector_target']

In [88]:
preprocessor = ColumnTransformer([
    ('cat_transformer', cat_transformer, cat_features)], remainder='passthrough')

In [89]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', LinearRegression())])

In [90]:
cross_val_score(pipe, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

-0.0025489291865800593

In [91]:
cross_val_score(pipe, X_train, y_train, cv=5, scoring='r2').mean()

-0.036103584870120865

# ElasticNet

In [92]:
grid = {'model__alpha': [0.2, 0.3, 0.4, 0.5, 0.6],
        'model__l1_ratio': [0,0.2],
        'model__max_iter': [10000]}

In [93]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', ElasticNet())])

In [94]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = ['neg_mean_squared_error', 'r2','neg_mean_absolute_error'],
                      refit='neg_mean_squared_error',
                      cv = 5,
                      n_jobs=-1) 

In [95]:
# Fit data to Grid Search
search.fit(X_train,y_train)

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.783448621372009, tolerance: 0.0005684619764424259
  model = cd_fast.sparse_enet_coordinate_descent(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat_transformer',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['target_status',
                                                                          'shares_at_announcement',
                                                                          'shares_acquired',
                                                                          'consideration_offered',
                                                                          'cross_border',
                                                                          'relatedness',
                                                                          

In [96]:
# Best score 1
search.best_score_

-0.002438774561070367

In [97]:
search.best_params_

{'model__alpha': 0.3, 'model__l1_ratio': 0, 'model__max_iter': 10000}

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.257856837677321, tolerance: 0.000465706556622184
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.2678948976052036, tolerance: 0.000465706556622184
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.173786069287569, tolerance: 0.000445070853803